In [0]:
# Importações necessárias
import os  # Não está sendo usado no código — pode ser removido se não for necessário futuramente
from pyspark.sql.utils import AnalysisException  # Importação específica que também não está sendo usada diretamente

# Caminho onde estão os arquivos .parquet
volume_path = "/Volumes/ted_dev/dbt_adventureworks_raw/raw"

# Validação da estrutura do caminho e extração de catálogo e schema
try:
    path_parts = volume_path.strip("/").split("/")  # Divide o caminho em partes ignorando barras iniciais/finais
    if len(path_parts) < 4 or path_parts[0] != "Volumes":  # Verifica se o caminho tem estrutura esperada
        raise ValueError("Invalid path")  # Erro se estrutura estiver incorreta
    catalog_name = path_parts[1]  # Nome do catálogo
    schema_name = path_parts[2]   # Nome do schema
except Exception as e:
    raise ValueError(f"error")  # Exceção genérica — seria melhor incluir a mensagem do erro

# Listagem dos arquivos dentro do volume especificado
try:
    all_files = dbutils.fs.ls(volume_path)  # Lista arquivos e diretórios no caminho
except Exception as e:
    raise Exception(f"Error")  # Exceção genérica — poderia ser mais informativa

# Iteração sobre os arquivos encontrados
for file_info in all_files:

    # Verifica se o item é um arquivo (não diretório) e se é um arquivo .parquet
    if not file_info.isDir() and file_info.name.endswith(".parquet"):

        source_file_path = file_info.path  # Caminho completo do arquivo .parquet
        table_name = file_info.name.removesuffix(".parquet")  # Nome da tabela extraído do nome do arquivo

        try:
            df = spark.read.parquet(source_file_path)  # Leitura do arquivo .parquet em um DataFrame Spark

            # Monta o nome completo da tabela usando catálogo, schema e nome da tabela
            full_table_name = f"`{catalog_name}`.`{schema_name}`.`{table_name}`"

            # Escrita do DataFrame como tabela no metastore usando modo overwrite
            df.write.mode("overwrite").saveAsTable(full_table_name)

        except Exception as e:
            print(f"error")  # Exceção genérica — poderia logar detalhes do erro para facilitar depuração